## 1. 偏函数 functools.partial

> 偏函数是2.5版本以后引进来的东西。属于函数式编程的一部分，使用偏函数可以通过有效地“冻结”那些预先确定的参数，来缓存函数参数，然后在运行时，当获得需要的剩余参数后，可以将他们解冻，传递到最终的参数中，从而使用最终确定的所有参数去调用函数。

In [1]:
from functools import partial

def add_func(*args):
    res = 0 
    for i in args:
        res += i
    return res

In [2]:
plus = partial(add_func, 10)
plus(10,20)

40

定义一个函数add_func(*awg)，实现对所有的位置参数累加  
当知道每次调用add_func时，必会加10，避免多次调用反复使用相同参数的情况，使用偏函数把10绑定，多次调用的时候就减少了重复。
***

## 2. 闭包

从下面代码找找感觉

In [3]:
def outer():
    x = 1
    def inner():
        print(x) # 1
    return inner

foo = outer()

foo()

1


inner作为函数的引用由outer返回，保存在foo变量，foo指向函数inner的引用，且foo()可以成功打印x的值为1。  
这是因为Python支持一个叫做函数闭包的特性，通俗讲：嵌套定义在非全局作用域里面的函数能够记住它在被定义的时候它所处的封闭命名空间。

**每次函数outer被调用的时候，函数inner都会被重新定义。**  
对上面代码稍稍改动：

In [4]:
def outer(x):
    def inner():
        print(x) # 1
    return inner

fun1 = outer(1)
fun2 = outer(2)

fun1()
fun2()

1
2


**闭包 -- 被函数记住的封闭作用域**  
***

## 3. 装饰器

> 装饰器本质上是一个Python函数，它可以让其他函数在不需要做任何代码变动的前提下增加额外功能，装饰器的返回值也是一个函数对象。它经常用于有切面需求的场景，比如：插入日志、性能测试、事务处理、缓存、权限校验等场景。装饰器是解决这类问题的绝佳设计，有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码并继续重用。

一句话概括，装饰器的作用就是，在不改变已存在的函数或对象前提下，为其添加额外的功能

In [5]:
from time import ctime, sleep

def timefun(func):
    def wrapped_func():
        print("%s called at %s"%(func.__name__, ctime()))
        func()
    return wrapped_func

@timefun
def foo():
    print("I am foo")

foo()
sleep(2)
foo()

foo called at Mon Jul 23 10:45:37 2018
I am foo
foo called at Mon Jul 23 10:45:39 2018
I am foo


@timefun 就是使用了装饰器，相当于foo = timefun(foo)
foo先作为参数赋值给func后,foo接收指向timefun返回的wrapped_func
foo()
调用foo(),即等价调用wrapped_func()
内部函数wrapped_func被引用，所以外部函数的func变量(自由变量)并没有释放
func里保存的是原foo函数对象

### 装饰器功能

- 引用日志
- 函数执行时间统计
- 执行函数前预备处理
- 执行函数后清理处理
- 权限校验
- 缓存

装饰器示例  

** *1. 无参数的装饰器（上面例子即是）**

** *2. 被修饰的函数有参数**

In [6]:
def timefun(func):
    def wrapped_func(a, b):
        print('{} called at {}'.format(func.__name__, ctime()))
        print(a, b)
        func(a, b)
    return wrapped_func

@timefun
def foo(a, b):
    print(a + b)
    
foo(3, 5)
sleep(1)
foo(1, 2)

foo called at Mon Jul 23 10:45:39 2018
3 5
8
foo called at Mon Jul 23 10:45:40 2018
1 2
3


** *3. 被修饰的函数有不定长参数**

In [7]:
def timefun(func):
    def wrapped_func(*args, **kwargs):
        print("%s called at %s"%(func.__name__, ctime()))
        func(*args, **kwargs)
    return wrapped_func

@timefun
def foo(a, b, c):
    print(a+b+c)

foo(3,5,7)
sleep(2)
foo(2,4,9)

foo called at Mon Jul 23 10:45:40 2018
15
foo called at Mon Jul 23 10:45:42 2018
15


** *4. 装饰器中的return**

In [8]:
def timefun(func):
    def wrapped_func(*args, **kwargs):
        print("%s called at %s"%(func.__name__, ctime()))
        return func(*args, **kwargs)
    return wrapped_func
@timefun
def get_info():
    return '-----hahahaha------'

get_info()

get_info called at Mon Jul 23 10:45:42 2018


'-----hahahaha------'

** *5. 装饰器带有参数，在原有装饰器的基础上，设置外部变量**

In [9]:
def timefun_arg(pre="hello"):
    def timefun(func):
        def wrappedfunc():
            print("%s called at %s %s"%(func.__name__, ctime(), pre))
            return func()
        return wrappedfunc
    return timefun

@timefun_arg("itcast")
def foo():
    print("I am foo")

@timefun_arg("python")
def too():
    print("I am too")

foo()
sleep(1)
foo()

too()
sleep(1)
too()

foo called at Mon Jul 23 10:45:42 2018 itcast
I am foo
foo called at Mon Jul 23 10:45:43 2018 itcast
I am foo
too called at Mon Jul 23 10:45:43 2018 python
I am too
too called at Mon Jul 23 10:45:44 2018 python
I am too


**可以理解为  
foo()==timefun_arg("itcast")(foo)()**

** *6. 类装饰器**

装饰器函数其实是这样一个接口约束，它必须接受一个callable对象作为参数，然后返回一个callable对象。在Python中一般callable对象都是函数，但也有例外。只要某个对象重写了 __call__() 方法，那么这个对象就是callable的。

In [10]:
class Test():
    def __call__(self):
        print('call me!')

t = Test()
t()  # call me

call me!


类装饰器 demo

In [11]:
class Test(object):
    def __init__(self, func):
        print("---初始化---")
        print("func name is %s"%func.__name__)
        self.__func = func
    def __call__(self):
        print("---装饰器中的功能---")
        self.__func()

@Test
def test():
    print("----test---")

test()

---初始化---
func name is test
---装饰器中的功能---
----test---


说明：
1. 当用Test来装作装饰器对test函数进行装饰的时候，首先会创建Test的实例对象
并且会把test这个函数名当做参数传递到\__init\__方法中
即在\__init__方法中的func变量指向了test函数体
2. test函数相当于指向了用Test创建出来的实例对象
3. 当在使用test()进行调用时，就相当于让这个对象()，因此会调用这个对象的\__call__方法
4. 为了能够在\__call__方法中调用原来test指向的函数体，所以在\__init__方法中就需要一个实例属性来保存这个函数体的引用
所以才有了self.\__func = func这句代码，从而在调用\__call__方法中能够调用到test之前的函数体

- 小细节——wraps函数

使用装饰器时，有一些细节需要被注意。例如，被装饰后的函数其实已经是另外一个函数了（函数名等函数属性会发生改变）。  
添加后由于函数名和函数的doc发生了改变，对测试结果有一些影响，例如:

In [12]:
def note(func):
    "note function"
    def wrapper():
        "wrapper function"
        print('note something')
        return func()
    return wrapper

@note
def test():
    "test function"
    print('I am test')

test()
print(test.__doc__)

note something
I am test
wrapper function


所以，Python的functools包中提供了一个叫wraps的装饰器来消除这样的副作用。例如：


In [13]:
import functools
def note(func):
    "note function"
    @functools.wraps(func)
    def wrapper():
        "wrapper function"
        print('note something')
        return func()
    return wrapper

@note
def test():
    "test function"
    print('I am test')

test()
print(test.__doc__)

note something
I am test
test function
